### INSTALL DEPENDENCIES

In [ ]:
! git clone https://github.com/ultralytics/yolov5

In [5]:
! cd yolov5 & pip install -r requirements.txt

     -------------------------------------- 611.8/611.8 kB 9.7 MB/s eta 0:00:00
     -------------------------------------- 805.2/805.2 kB 7.3 MB/s eta 0:00:00
     -------------------------------------- 172.4/172.4 MB 4.4 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 63.4.1
    Uninstalling setuptools-63.4.1:
      Successfully uninstalled setuptools-63.4.1
  Attempting uninstall: torch
    Found existing installation: torch 1.8.1+cu111
    Uninstalling torch-1.8.1+cu111:
      Successfully uninstalled torch-1.8.1+cu111


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.2.2 requires pyqt5<5.13, which is not installed.
spyder 5.2.2 requires pyqtwebengine<5.13, which is not installed.
conda-repo-cli 1.0.41 requires requests_mock, which is not installed.
anaconda-project 0.11.1 requires ruamel-yaml, which is not installed.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.23.5 which is incompatible.
conda-repo-cli 1.0.41 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.41 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.


In [2]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [6]:
import uuid  #unique identifier
import os
import time

In [11]:
IMAGES_PATH = os.path.join('data','images')
labels = ['cap', 'withoutcap']
number_images = 20

### COLLECT IMAGES FROM WEBCAM

In [12]:
# Capturing the images for training the yolo model

cap = cv2.VideoCapture(0)
# loop through labels
for label in labels:
    print('Collecting images for {}'.format(label))
    time.sleep(5)

    # loop through images
    for img_num in range(number_images):
        print('Collecting images for {}, image number {}'.format(label,img_num))

        # webcam feed
        ret, frame = cap.read()

        # naming out image path
        imgname = os.path.join(IMAGES_PATH, label+'.'+str(uuid.uuid1())+'.jpg')

        # writes image to file
        cv2.imwrite(imgname, frame)
        
        # render to the screen
        cv2.imshow('Image collection', frame)
        time.sleep(2)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()


### TO LABEL THE DATA

In [13]:
# for data labeling
! git clone https://github.com/tzutalin/labelImg

Cloning into 'labelImg'...


In [ ]:
! pip install pyqt5 lxml --upgrade
! cd labelImg && pyrcc5 -o libs/resources.py resources.qrc

### TRAINING THE MODEL WITH CAP AND WITHOUT CAP DATA

In [16]:
!cd yolov5 && python train.py --img 320 --batch 16 --epochs 50 --data dataset.yml --weights yolov5s.pt --workers 2


^C


In [4]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp/weights/last.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\hrush/.cache\torch\hub\master.zip
YOLOv5  2023-8-18 Python-3.9.13 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7055974 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


In [7]:
img = os.path.join('data', 'images', 'cap.8ea073db-3d25-11ee-950e-346f24c688f6.jpg')

In [8]:
results = model(img)

In [9]:
results.print()

image 1/1: 480x640 (no detections)
Speed: 18.1ms pre-process, 257.1ms inference, 0.0ms NMS per image at shape (1, 3, 480, 640)


### TESTING THE MODEL

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()

    results = model(frame)

    cv2.imshow('YOLO', np.squeeze(results.render()))

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()